In [2]:
# =========================================================
# Cell 1: Load Diabetes Meal Dataset
# =========================================================
import pandas as pd
import numpy as np

# Load your file
file_path = "/Users/kshitizsikriwal/Kshitiz/evaluation/Gemini/meal_to_yoga_top1_contrastive_diabetes.xlsx"
df = pd.read_excel(file_path)

# Keep only relevant columns
cols = [
    "meal_plan", "meal_day", "meal_time", "meal_name", "meal_description",
    "Calories (kcal)", "Total_Fat (g)", "Protein (g)", "Carbohydrates (g)",
    "Dietary_Fiber (g)", "Added_Sugar (g)", "Omega-3 (ALA, g)", 
    "EPA + DHA (mg)", "Selenium (µg)", "Magnesium (mg)"
]
df = df[cols]

print("✅ Dataset loaded successfully — shape:", df.shape)
df.head(3)


✅ Dataset loaded successfully — shape: (42, 15)


,meal_plan,meal_day,meal_time,meal_name,meal_description,Calories (kcal),Total_Fat (g),Protein (g),Carbohydrates (g),Dietary_Fiber (g),Added_Sugar (g),"Omega-3 (ALA, g)",EPA + DHA (mg),Selenium (µg),Magnesium (mg)
0,Plan A,Monday,Morning,Steel-Cut Oats,"Oats with flaxseeds, walnuts, and blueberries.",500,20,15,70,10,5,1.0,0,10,100
1,Plan A,Monday,Afternoon,Quinoa & Black Bean Salad,"Quinoa, black beans, corn, bell peppers, avoca...",700,25,25,100,15,5,0.5,0,15,150
2,Plan A,Monday,Evening,Grilled Salmon,Grilled salmon with roasted broccoli and a sma...,800,30,50,75,10,5,0.1,300,30,80


In [3]:
# =========================================================
# Cell 2: Clean and standardize column names
# =========================================================
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("(", "")
    .str.replace(")", "")
    .str.replace("+", "plus")
    .str.replace("µ", "u")
    .str.replace("-", "_")
)

weekday_order = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
df["meal_day"] = df["meal_day"].astype(str).str.strip().str.lower()
df["meal_day"] = pd.Categorical(df["meal_day"], categories=weekday_order, ordered=True)

print("✅ Columns standardized:")
print(df.columns.tolist())
df.head(3)


✅ Columns standardized:
['meal_plan', 'meal_day', 'meal_time', 'meal_name', 'meal_description', 'calories_kcal', 'total_fat_g', 'protein_g', 'carbohydrates_g', 'dietary_fiber_g', 'added_sugar_g', 'omega_3_ala,_g', 'epa_plus_dha_mg', 'selenium_ug', 'magnesium_mg']


,meal_plan,meal_day,meal_time,meal_name,meal_description,calories_kcal,total_fat_g,protein_g,carbohydrates_g,dietary_fiber_g,added_sugar_g,"omega_3_ala,_g",epa_plus_dha_mg,selenium_ug,magnesium_mg
0,Plan A,monday,Morning,Steel-Cut Oats,"Oats with flaxseeds, walnuts, and blueberries.",500,20,15,70,10,5,1.0,0,10,100
1,Plan A,monday,Afternoon,Quinoa & Black Bean Salad,"Quinoa, black beans, corn, bell peppers, avoca...",700,25,25,100,15,5,0.5,0,15,150
2,Plan A,monday,Evening,Grilled Salmon,Grilled salmon with roasted broccoli and a sma...,800,30,50,75,10,5,0.1,300,30,80


In [4]:
# =========================================================
# Cell 3: Define daily recommended ranges for diabetes
# =========================================================
nutrient_ranges = {
    "calories_kcal": (1800, 2200),
    "total_fat_g": (44, 78),
    "protein_g": (50, 120),
    "carbohydrates_g": (180, 275),
    "dietary_fiber_g": (25, 40),
    "added_sugar_g": (0, 25),
    "omega_3_alag": (1.1, 1.6),
    "epa_plus_dhamg": (250, 500),
    "selenium_ug": (55, 55),
    "magnesium_mg": (310, 420)
}

ranges_df = pd.DataFrame([
    {"Nutrient": k, "Min_Recommended": v[0], "Max_Recommended": v[1]}
    for k, v in nutrient_ranges.items()
])

print("✅ Reference nutrient ranges for adults with Diabetes:")
display(ranges_df)


✅ Reference nutrient ranges for adults with Diabetes:


,Nutrient,Min_Recommended,Max_Recommended
0,calories_kcal,1800.0,2200.0
1,total_fat_g,44.0,78.0
2,protein_g,50.0,120.0
3,carbohydrates_g,180.0,275.0
4,dietary_fiber_g,25.0,40.0
5,added_sugar_g,0.0,25.0
6,omega_3_alag,1.1,1.6
7,epa_plus_dhamg,250.0,500.0
8,selenium_ug,55.0,55.0
9,magnesium_mg,310.0,420.0


In [5]:
# =========================================================
# Cell 4: Manually map nutrient keys to dataset columns (robust)
# =========================================================
# initial desired mapping (logical nutrient keys -> expected column names)
col_map = {
    "calories_kcal": "calories_kcal",
    "total_fat_g": "total_fat_g",
    "protein_g": "protein_g",
    "carbohydrates_g": "carbohydrates_g",
    "dietary_fiber_g": "dietary_fiber_g",
    "added_sugar_g": "added_sugar_g",
    "omega_3_alag": "omega_3_alag",
    "epa_plus_dhamg": "epa_plus_dhamg",
    "selenium_ug": "selenium_ug",
    "magnesium_mg": "magnesium_mg"
}

# helper to normalize names for matching
def _norm(s):
    return (
        str(s).lower()
        .replace(" ", "")
        .replace("_", "")
        .replace("(", "")
        .replace(")", "")
        .replace(",", "")
        .replace("+", "plus")
        .replace("µ", "u")
        .replace("-", "")
        .replace(".", "")
    )

# build normalized lookup of actual dataframe columns
norm_to_col = {_norm(c): c for c in df.columns}

resolved_map = {}
unresolved = []

for key, desired in col_map.items():
    # 1) exact match
    if desired in df.columns:
        resolved_map[key] = desired
        continue

    # 2) normalized desired match
    n_desired = _norm(desired)
    if n_desired in norm_to_col:
        resolved_map[key] = norm_to_col[n_desired]
        continue

    # 3) normalized key match (e.g., 'omega_3_alag' -> contains 'omega3' / 'ala' etc.)
    n_key = _norm(key)
    matches = [col for norm, col in norm_to_col.items() if n_key in norm]
    if len(matches) == 1:
        resolved_map[key] = matches[0]
        continue

    # 4) try partial matching by requiring all meaningful parts to appear
    parts = [p for p in key.split("_") if p]
    candidates = [
        col
        for norm, col in norm_to_col.items()
        if all(part in norm for part in parts)
    ]
    if len(candidates) == 1:
        resolved_map[key] = candidates[0]
        continue

    # if still not found, mark unresolved
    unresolved.append(key)

# common manual fixes for known naming differences in this dataset
manual_fixes = {
    "omega_3_alag": "omega_3_ala,_g",
    "epa_plus_dhamg": "epa_plus_dha_mg"
}
for k, v in manual_fixes.items():
    if k in unresolved and v in df.columns:
        resolved_map[k] = v
        unresolved.remove(k)

if unresolved:
    print("⚠️ Could not resolve columns for keys:", unresolved)
    print("Available columns:", df.columns.tolist())

# Convert to numeric safely using resolved_map values
for colname in set(resolved_map.values()):
    df[colname] = pd.to_numeric(df[colname], errors="coerce")

print("✅ Column mapping resolved:")
for k, v in resolved_map.items():
    print(f"{k:20} --> {v}")

# update col_map so downstream code uses the resolved mapping
col_map = resolved_map


✅ Column mapping resolved:
calories_kcal        --> calories_kcal
total_fat_g          --> total_fat_g
protein_g            --> protein_g
carbohydrates_g      --> carbohydrates_g
dietary_fiber_g      --> dietary_fiber_g
added_sugar_g        --> added_sugar_g
omega_3_alag         --> omega_3_ala,_g
epa_plus_dhamg       --> epa_plus_dha_mg
selenium_ug          --> selenium_ug
magnesium_mg         --> magnesium_mg


In [6]:
# =========================================================
# Cell 5: Aggregate nutrient totals per day and plan
# =========================================================
agg_cols = list(col_map.values())
daily = df.groupby(["meal_plan", "meal_day"], as_index=False)[agg_cols].sum()

print("✅ Daily aggregated nutrient totals:")
display(daily.head())


✅ Daily aggregated nutrient totals:


/var/folders/jg/8wn2439s6y312d5jyqw78pc80000gn/T/ipykernel_43052/1755986064.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  daily = df.groupby(["meal_plan", "meal_day"], as_index=False)[agg_cols].sum()


,meal_plan,meal_day,calories_kcal,total_fat_g,protein_g,carbohydrates_g,dietary_fiber_g,added_sugar_g,"omega_3_ala,_g",epa_plus_dha_mg,selenium_ug,magnesium_mg
0,Plan A,monday,2000,75,90,245,35,15,1.6,300,55,330
1,Plan A,tuesday,2000,70,120,215,30,11,1.3,150,55,340
2,Plan A,wednesday,2000,78,100,225,37,16,1.6,350,55,380
3,Plan A,thursday,2000,75,100,220,40,17,1.2,250,55,350
4,Plan A,friday,1950,52,115,220,30,12,1.6,330,55,340


In [7]:
# =========================================================
# Cell 6: Show sample comparisons for verification
# =========================================================
plan_example = daily[daily["meal_plan"].str.contains("a", case=False)].head(3)
compare_table = []

for nutrient, (low, high) in nutrient_ranges.items():
    col = col_map[nutrient]
    for _, r in plan_example.iterrows():
        val = round(float(r[col]), 2)
        compare_table.append({
            "Meal_Plan": r["meal_plan"],
            "Meal_Day": r["meal_day"],
            "Nutrient": nutrient,
            "Generated_Value": val,
            "Target_Range": f"{low} - {high}"
        })

compare_df = pd.DataFrame(compare_table)
display(compare_df)


,Meal_Plan,Meal_Day,Nutrient,Generated_Value,Target_Range
0,Plan A,monday,calories_kcal,2000.0,1800 - 2200
1,Plan A,tuesday,calories_kcal,2000.0,1800 - 2200
2,Plan A,wednesday,calories_kcal,2000.0,1800 - 2200
3,Plan A,monday,total_fat_g,75.0,44 - 78
4,Plan A,tuesday,total_fat_g,70.0,44 - 78
5,Plan A,wednesday,total_fat_g,78.0,44 - 78
6,Plan A,monday,protein_g,90.0,50 - 120
7,Plan A,tuesday,protein_g,120.0,50 - 120
8,Plan A,wednesday,protein_g,100.0,50 - 120
9,Plan A,monday,carbohydrates_g,245.0,180 - 275


In [8]:
# =========================================================
# Cell 7: Evaluate TRUE/FALSE for strict and partial compliance
# =========================================================
def evaluate_compliance(value, low, high, tolerance=0.05):
    if pd.isna(value):
        return np.nan, np.nan
    strict = low <= value <= high
    partial = (low * (1 - tolerance)) <= value <= (high * (1 + tolerance))
    return strict, partial

records = []
for _, row in daily.iterrows():
    record = {"meal_plan": row["meal_plan"], "meal_day": row["meal_day"]}
    for nutrient, (low, high) in nutrient_ranges.items():
        col = col_map[nutrient]
        val = row[col]
        strict, partial = evaluate_compliance(val, low, high)
        record[f"{nutrient}_value"] = round(val, 2)
        record[f"{nutrient}_strict"] = strict
        record[f"{nutrient}_partial"] = partial
    records.append(record)

compliance_df = pd.DataFrame(records)
display(compliance_df.head())


,meal_plan,meal_day,calories_kcal_value,calories_kcal_strict,calories_kcal_partial,total_fat_g_value,total_fat_g_strict,total_fat_g_partial,protein_g_value,protein_g_strict,...,omega_3_alag_partial,epa_plus_dhamg_value,epa_plus_dhamg_strict,epa_plus_dhamg_partial,selenium_ug_value,selenium_ug_strict,selenium_ug_partial,magnesium_mg_value,magnesium_mg_strict,magnesium_mg_partial
0,Plan A,monday,2000,True,True,75,True,True,90,True,...,True,300,True,True,55,True,True,330,True,True
1,Plan A,tuesday,2000,True,True,70,True,True,120,True,...,True,150,False,False,55,True,True,340,True,True
2,Plan A,wednesday,2000,True,True,78,True,True,100,True,...,True,350,True,True,55,True,True,380,True,True
3,Plan A,thursday,2000,True,True,75,True,True,100,True,...,True,250,True,True,55,True,True,350,True,True
4,Plan A,friday,1950,True,True,52,True,True,115,True,...,True,330,True,True,55,True,True,340,True,True


In [9]:
# =========================================================
# Cell 8: Count compliant nutrients (strict + partial)
# =========================================================
strict_cols = [c for c in compliance_df.columns if "_strict" in c]
partial_cols = [c for c in compliance_df.columns if "_partial" in c]

compliance_df["strict_count"] = compliance_df[strict_cols].sum(axis=1)
compliance_df["partial_count"] = compliance_df[partial_cols].sum(axis=1)
compliance_df["total_nutrients"] = len(strict_cols)

compliance_df["strict_compliance_%"] = round(
    compliance_df["strict_count"] / compliance_df["total_nutrients"] * 100, 2)
compliance_df["partial_compliance_%"] = round(
    compliance_df["partial_count"] / compliance_df["total_nutrients"] * 100, 2)

print("✅ Nutrient compliance count per plan/day:")
display(compliance_df[
    ["meal_plan", "meal_day", "strict_count", "partial_count",
     "total_nutrients", "strict_compliance_%", "partial_compliance_%"]
])


✅ Nutrient compliance count per plan/day:


,meal_plan,meal_day,strict_count,partial_count,total_nutrients,strict_compliance_%,partial_compliance_%
0,Plan A,monday,10,10,10,100.0,100.0
1,Plan A,tuesday,9,9,10,90.0,90.0
2,Plan A,wednesday,10,10,10,100.0,100.0
3,Plan A,thursday,10,10,10,100.0,100.0
4,Plan A,friday,10,10,10,100.0,100.0
5,Plan A,saturday,10,10,10,100.0,100.0
6,Plan A,sunday,8,8,10,80.0,80.0
7,Plan B,monday,10,10,10,100.0,100.0
8,Plan B,tuesday,9,10,10,90.0,100.0
9,Plan B,wednesday,9,9,10,90.0,90.0


In [10]:
# =========================================================
# Cell 9: Compute average compliance per plan
# =========================================================
summary = (
    compliance_df.groupby("meal_plan")[["strict_compliance_%", "partial_compliance_%"]]
    .mean()
    .reset_index()
)

summary.rename(columns={
    "strict_compliance_%": "Avg_Strict_Compliance(%)",
    "partial_compliance_%": "Avg_Partial_Compliance(%)"
}, inplace=True)

print("✅ Average compliance summary per plan:")
display(summary)


✅ Average compliance summary per plan:


,meal_plan,Avg_Strict_Compliance(%),Avg_Partial_Compliance(%)
0,Plan A,95.714286,95.714286
1,Plan B,92.857143,94.285714


In [11]:
# =========================================================
# Cell 10: Generate storytelling output for research results
# =========================================================
stories = []
for _, row in summary.iterrows():
    plan = row["meal_plan"]
    s = row["Avg_Strict_Compliance(%)"]
    p = row["Avg_Partial_Compliance(%)"]

    story = (
        f"For {plan}, mean strict compliance was {s:.1f}% "
        f"and partial compliance {p:.1f}%. "
        f"This indicates that, on average, {s:.1f}% of the 10 evaluated nutrients "
        f"met the strict recommended range, while {p:.1f}% were within ±5% tolerance."
    )
    stories.append(story)

print("🧾 Narrative Summary for Adults with Diabetes:\n")
for s in stories:
    print(s)


🧾 Narrative Summary for Adults with Diabetes:

For Plan A, mean strict compliance was 95.7% and partial compliance 95.7%. This indicates that, on average, 95.7% of the 10 evaluated nutrients met the strict recommended range, while 95.7% were within ±5% tolerance.
For Plan B, mean strict compliance was 92.9% and partial compliance 94.3%. This indicates that, on average, 92.9% of the 10 evaluated nutrients met the strict recommended range, while 94.3% were within ±5% tolerance.
